### Title: Browse multiple molecules
# Description: browsing multiple molecules, each of which has multiple conformations
Derived from Gerg's code by Malitha 
(Updated: 23th June 2017)

In [12]:
import os
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
from time import time

# Definition: Default Rendering Style

In [13]:
global myGlobalStyle
molSize_3d = (400, 400)
myGlobalStyle = 'stick' # default drawing type for 3d structures # line cross stick cartoon sphere
bgcolor_3d = '0xeeeeee'

 

# Generate Conformers

In [14]:
data = [('m1','COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m2','COc1ccc2[nH]c([S@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m3','COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m4','CCOc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OCCC)c3C)nc2c1')]
ms = [(x,Chem.AddHs(Chem.MolFromSmiles(y))) for x,y in data]
params = AllChem.ETKDG()
params.numThreads=3
for nm,m in ms:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)
    # align to one of the ring systems:
    AllChem.AlignMolConformers(m,m.GetSubstructMatch(Chem.MolFromSmarts('c1[nH]c2ccccc2n1')))


# Required functions for scrolling capability

In [15]:

def startViewer(size=None,bgcolor=None):
    if size is None:
        size=molSize_3d
    if bgcolor is None:
        bgcolor=bgcolor_3d
    view = py3Dmol.view(width=size[0],height=size[1])
    view.setBackgroundColor(bgcolor)
    return view


def processSuppliedMolFor3DViewer(ms):
    
    try:
        # list of tuple (name + mol obj) # dict key possible
        # ms = [('m1', <rdkit.Chem.rdchem.Mol at 0x7246d88>),
        # ('m2', <rdkit.Chem.rdchem.Mol at 0x7246ea0>),
        # ('m3', <rdkit.Chem.rdchem.Mol at 0x7246ed8>),
        # ('m4', <rdkit.Chem.rdchem.Mol at 0x7246f10>)]
        moldict = dict(ms)
        
    except TypeError:
        
        if type(ms) is tuple:
            # Not list single tuple (name + mol obj) # dict key possible
            # ms = ('m1', <rdkit.Chem.rdchem.Mol at 0x7246d88>)
            moldict=list()
            moldict.append(ms)
            moldict = dict(moldict)
        elif hasattr(ms, '__iter__') is False:
            # rdkit.Chem.rdchem.Mol
            # Not list... mol obj only ... no name... no dict key possible from such list...
            # So, generate dict key
            # ms = <rdkit.Chem.rdchem.Mol object at 0x07246D88>
            moldict=list()
            moldict.append(('m0', ms))
            moldict = dict(moldict)
        elif type(ms) is list:
            # list of mol obj only ... no name... no dict key possible from such list...
            # So, generate dict key
            # ms = [<rdkit.Chem.rdchem.Mol at 0x7246d88>,
            # <rdkit.Chem.rdchem.Mol at 0x7246ea0>,
            # <rdkit.Chem.rdchem.Mol at 0x7246ed8>,
            # <rdkit.Chem.rdchem.Mol at 0x7246f10>]
            ms_name=['m'+str(x) for x in range(len(ms))]
            ms2=[(ms_name[i],ms[i]) for i in range(len(ms))]
            moldict = dict(ms2)
    return moldict
    
def addMolToViewForScrolling(category,mols,view,confId,
                             useDrawAs,drawAs, 
                             showConfLabel, 
                             showAtomLabel,
                             molColorScheme):
    
    mol = mols[category]
    conf_selected=mol.GetConformer(confId)
    xyz=conf_selected.GetPositions()
    
    view.removeAllModels()
    view.removeAllSurfaces()
    view.removeAllLabels()
    
    if mol.GetNumAtoms()>=999 or drawAs == 'cartoon':
        # py3DMol is happier with TER and MASTER records present
        pdb = Chem.MolToPDBBlock(mol,flavor=0x20|0x10)
        view.addModel(pdb,'pdb')
    else:
        # py3Dmol does not currently support v3k mol files, so
        # we can only provide those with "smaller" molecules
        mb = Chem.MolToMolBlock(mol,confId=confId)
        view.addModel(mb,'sdf')
        
        
    global myGlobalStyle    
    if useDrawAs is False:
        #use from globalStyle
        view.setStyle({},{myGlobalStyle:{'colorscheme': molColorScheme}})
    else:
        #update global style and use that
        myGlobalStyle = drawAs
        view.setStyle({},{myGlobalStyle:{'colorscheme': molColorScheme}})
    
    # This is exception for surface
    if drawAs is 'surface':
        view.addSurface({}, '$3Dmol.SurfaceType.VDW');
        
    if drawAs is 'ballstick':
        view.setStyle({},{'stick':{'radius':'0.2','colorscheme': molColorScheme},
                          'sphere':{'radius':'0.4', 'colorscheme': molColorScheme}}
                     );
    
    if showConfLabel is True:
        label=category+':'+str(confId)
        view.addLabel(label, {'backgroundColor':'gray', 'fontColor':'white',
                              'showBackground':'true', 'alignment':'bottomCenter'})
        #view.addLabel(label, {'alignment':'topCenter','backgroundColor':'red'})
    
    if showAtomLabel is True:
        
        OwningMol=conf_selected.GetOwningMol()
        label_create=[OwningMol.GetAtomWithIdx(i).GetSymbol()+
                      str(OwningMol.GetAtomWithIdx(i).GetIdx()+1) 
                      for i in range(conf_selected.GetNumAtoms())
                     ]
        
        i = None
        for i in range(conf_selected.GetNumAtoms()):
            view.addLabel(label_create[i], {'inFront' : 'false', 
                                            'fontSize' : '12',
                                            'fontColor':'gray',
                                            'showBackground':'false',
                                            'position' : {'x' : xyz[i][0],
                                                          'y' : xyz[i][1],
                                                          'z' : xyz[i][2]
                                                       }
                                           })
    
    #print(drawAs)
    # zoomTo does not work well for surface and label... so, zoomTo should not be default settings
    #view.zoomTo()
    return view.update()


def browseMolConformers(ms,view, confId=None, useDrawAs=False, drawAs=None, showConfLabel=None, showAtomLabel=None):
    
    drawing_type_3d=['line', 'cross', 'stick', 'cartoon', 'sphere', 'surface', 'ballstick']
    color_scheme_3d=['default', 'greenCarbon', 'cyanCarbon', 'magentaCarbon', 
                     'yellowCarbon', 'whiteCarbon', 'orangeCarbon', 'purpleCarbon', 
                     'blueCarbon', 'ssPyMOL', 'ssJmol', 'Jmol', 'amino', 
                     'shapely', 'nucleic', 'chain', 'chainHetatm', 'prop']

    global myGlobalStyle
    
    if useDrawAs is False:
        # Then go with default settings
        drawAs = myGlobalStyle
        style_use_logical = widgets.Dropdown(options=[False, True],value=False)
        
        # To do: showing massage is argument mismatched
        # Something like the following
        #if drawAs is not None:
            #print('drawAs set to stick as you choose useDrawAs=False')
            
        
    else:
        style_use_logical = widgets.Dropdown(options=[False, True],value=True)
        # Use user supplied drawAS
        if drawAs is None:
            # User forgot to provide drawAs argument
            drawAs = myGlobalStyle
        else:
            # User supplied drawAs argument while useDrawAs is True
            myGlobalStyle = drawAs
        
    
    # This is widget for model style
    style_widget = widgets.Dropdown(options=drawing_type_3d,value=drawAs)
    
    # This is widget for conformer label
    if showConfLabel is not True:
        mol_labeling_widget = widgets.Checkbox(value=False)
    else:
        mol_labeling_widget = widgets.Checkbox(value=False)
    
    
    # This is widget for atom label of each conformers
    if showAtomLabel is not True:
        atom_labeling_widget = widgets.Checkbox(value=False)
    else:
        atom_labeling_widget = widgets.Checkbox(value=False)
    
    
    # processing supplied object that contains molecules
    moldict = processSuppliedMolFor3DViewer(ms)
    
    
    result=interact(addMolToViewForScrolling, 
                    category=list(moldict.keys()), 
                    mols=fixed(moldict),
                    view=fixed(view),
                    confId=confId, 
                    useDrawAs=style_use_logical,
                    drawAs=style_widget,
                    showConfLabel=mol_labeling_widget,
                    showAtomLabel=atom_labeling_widget,
                    molColorScheme=color_scheme_3d);
    return result



# Browse mols and confs

In [17]:
view=startViewer(bgcolor='black')
#view=startViewer()
view.show()

In [11]:
#browseMolConformers(ms,view,confId=(0,m.GetNumConformers()-1),useDrawAs=True,drawAs='stick')
browseMolConformers(ms,view,confId=(0, 9),useDrawAs=True,drawAs='stick')

<function __main__.addMolToViewForScrolling>

In [7]:
view.zoomTo()
view.update()

In [144]:
view.setBackgroundColor('0xeeeeee')
view.update()

In [18]:
# Lable creating system
#mol1=ms[0][1].GetConformer(0)
#OwningMol=mol1.GetOwningMol()
#label_create=[OwningMol.GetAtomWithIdx(i).GetSymbol()+str(OwningMol.GetAtomWithIdx(i).GetIdx()+1) for i in range(mol1.GetNumAtoms())] 
#label_create

In [19]:
dict(ms)

{'m1': <rdkit.Chem.rdchem.Mol at 0x716a308>,
 'm2': <rdkit.Chem.rdchem.Mol at 0x716a340>,
 'm3': <rdkit.Chem.rdchem.Mol at 0x716a378>,
 'm4': <rdkit.Chem.rdchem.Mol at 0x716a3b0>}

In [20]:
ms2=[(y) for (x,y) in ms]

In [21]:
ms2

In [22]:
ms[0]

('m1', <rdkit.Chem.rdchem.Mol at 0x716a308>)

In [23]:
#dir(ms)
m.GetNumConformers()

10

In [24]:
type(ms[0])

tuple

In [25]:
print(ms[0])
print(ms[0][0])
print(ms[0][1])

('m1', <rdkit.Chem.rdchem.Mol object at 0x0716A308>)
m1


In [26]:
ms[0][1].GetNumConformers()

10

In [27]:
#dir(ms[0][1].GetConformer(-1))

In [28]:
#8.4849   -0.7612   -0.4130 C   0  0  0  0  0  0  0  0  0  0  0  0

In [29]:
#print(Chem.MolToMolBlock(ms[0][1],confId=1))